# Sentence Classification

In [1]:
# Built-in library
import re
import json
import logging
from typing import Any, Dict, List, Optional, Union
import logging
import warnings

# Standard imports
import numpy as np
import pandas as pd
from rich import print
import torch

# Visualization
import matplotlib.pyplot as plt


# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

In [2]:
def set_up_logger(delim: str = "::") -> Any:
    """This is used to create a basic logger."""

    format_ = f"[%(levelname)s] {delim} %(asctime)s {delim} %(message)s"
    logging.basicConfig(level=logging.INFO, format=format_)
    logger = logging.getLogger(__name__)
    return logger


# Global variable
logger = set_up_logger()


def load_data(*, filename: str, sep: str = ",") -> pd.DataFrame:
    """This is used to load the data.

    NB: Supported formats are 'csv' and 'parquet'.

    Params:
    -------
        filename (str): The filepath.
        sep (str, default=","): The separator. e.g ',', '\t', etc

    Returns:
    --------
        data (pd.DataFrame): The loaded dataframe.
    """
    data = (
        pd.read_csv(filename, sep=sep)
        if filename.split(".")[-1] == "csv"
        else pd.read_parquet(filename)
    )
    logger.info(f"Shape of data: {data.shape}\n")
    return data

### Prepare Data

```text
Create:
- Training data
- Validation data
- Test data

Labels
------
salary
gambling
loan
airtime
ussdTransactions
flightRisk
savingsAndInvestments
entertainment
spend
  - posSpend
  - atmSpend
  - mobileSpend
  - webSpend
```

In [3]:
fp: str = "./my_data/trans_data.parquet"

df: pd.DataFrame = load_data(filename=fp).rename(columns={"cleaned_labels": "labels"})

df.head()

[INFO] :: 2023-10-28 20:00:23,751 :: Shape of data: (500000, 5)



,date,description,amount,type,labels
0,2022-05-29,POS/WEB PMT T MODERNCOMMUNICATI 000104 2070849Y NG,20000.000000,D,spend
1,2022-03-19,POS/WEB PMT NULL LA NG,13100.000000,D,spend
2,2022-01-14,FGN ELECTRONIC MONEY TRANSFER LEVY,100.000000,D,spendOnTransfers
3,2022-02-19,Quantum USSD,20026.880859,D,ussd
4,2022-05-28,POS/WEB PMT BETWAY NG/1853277904 PSTK LANG,10000.000000,D,gambling


In [4]:
labels: dict[str, Any] = list(df["labels"].unique())
id2label: dict[str, Any] = {idx: key for idx, key in enumerate(labels)}
label2id: dict[str, Any] = {key: idx for idx, key in id2label.items()}

print(f"label2id: {label2id}")

print(f"id2label: {id2label}")

label2id: {'spend': 0, 'spendOnTransfers': 1, 'ussd': 2, 'gambling': 3, 'other': 4, 'airtime': 5, 'salaryEarner': 
6, 'loanRepayments': 7, 'flightRisk': 8, 'loanAmount': 9, 'entertainment': 10}

id2label: {0: 'spend', 1: 'spendOnTransfers', 2: 'ussd', 3: 'gambling', 4: 'other', 5: 'airtime', 6: 
'salaryEarner', 7: 'loanRepayments', 8: 'flightRisk', 9: 'loanAmount', 10: 'entertainment'}

In [5]:
# Map the labels
df["labels"] = df["labels"].map(label2id)

df.head()

,date,description,amount,type,labels
0,2022-05-29,POS/WEB PMT T MODERNCOMMUNICATI 000104 2070849Y NG,20000.000000,D,0
1,2022-03-19,POS/WEB PMT NULL LA NG,13100.000000,D,0
2,2022-01-14,FGN ELECTRONIC MONEY TRANSFER LEVY,100.000000,D,1
3,2022-02-19,Quantum USSD,20026.880859,D,2
4,2022-05-28,POS/WEB PMT BETWAY NG/1853277904 PSTK LANG,10000.000000,D,3


### Split Data

```text
Split the data into:
- Training data
- Validation data
- Test data
```

In [6]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict


RANDOM_STATE: int = 123
TEST_SIZE: float = 0.08

X_t, X_test = train_test_split(df, test_size=TEST_SIZE, random_state=RANDOM_STATE)

X_t.shape, X_test.shape

((460000, 5), (40000, 5))

In [7]:
X_train, X_validation = train_test_split(
    X_t, test_size=TEST_SIZE, random_state=RANDOM_STATE
)
X_train.shape, X_validation.shape, X_test.shape

((423200, 5), (36800, 5), (40000, 5))

In [8]:
# Create dataset objects
train_dataset: DatasetDict = Dataset.from_pandas(df=X_train)
validation_dataset: DatasetDict = Dataset.from_pandas(df=X_validation)
test_dataset: DatasetDict = Dataset.from_pandas(df=X_test)

train_dataset

Dataset({
    features: ['date', 'description', 'amount', 'type', 'labels', '__index_level_0__'],
    num_rows: 423200
})

In [9]:
# Create dataset dict object
dataset: DatasetDict = DatasetDict(
    {"train": train_dataset, "validation": validation_dataset, "test": test_dataset}
)

dataset

DatasetDict({
    train: Dataset({
        features: ['date', 'description', 'amount', 'type', 'labels', '__index_level_0__'],
        num_rows: 423200
    })
    validation: Dataset({
        features: ['date', 'description', 'amount', 'type', 'labels', '__index_level_0__'],
        num_rows: 36800
    })
    test: Dataset({
        features: ['date', 'description', 'amount', 'type', 'labels', '__index_level_0__'],
        num_rows: 40000
    })
})

In [10]:
# Remove column
dataset = dataset.remove_columns(["__index_level_0__"])
dataset

DatasetDict({
    train: Dataset({
        features: ['date', 'description', 'amount', 'type', 'labels'],
        num_rows: 423200
    })
    validation: Dataset({
        features: ['date', 'description', 'amount', 'type', 'labels'],
        num_rows: 36800
    })
    test: Dataset({
        features: ['date', 'description', 'amount', 'type', 'labels'],
        num_rows: 40000
    })
})

In [11]:
print(dataset.get("train")[0])

print(dataset.get("validation")[0])

print(dataset.get("test")[0])

{
    'date': '2022-05-06',
    'description': 'MOTUNRAYO TITILOPE FAYOMI-WILLIAMS/A',
    'amount': 3000.0,
    'type': 'C',
    'labels': 4
}

{
    'date': '2022-04-22',
    'description': 'TRF/Payment for NDK/FRM DASMAN PANAN MAAN TO  DANIEL GBASIKA ADEBOYE - 011',
    'amount': 12026.8798828125,
    'type': 'D',
    'labels': 1
}

{'date': '2021-12-14', 'description': 'ATM CASH WDL ROU', 'amount': 3000.0, 'type': 'D', 'labels': 0}

In [12]:
def lower_case_slower(example: dict[str, Any]) -> dict[str, Any]:
    """Convert the text to lowercase. It's a MUCH SLOWER version."""
    return {"description": example.get("description").lower()}


def lower_case(example: dict[str, Any]) -> dict[str, Any]:
    """Convert the text to lowercase."""
    return {"description": [val.lower() for val in example.get("description")]}


def add_description_length(example: dict[str, Any]) -> dict[str, Any]:
    """This is used to add the length of the description to the dataset."""
    return {"description_length": [len(val) for val in example.get("description")]}

In [13]:
# Much slower
# dataset_1 = dataset.map(lower_case_slower)
# dataset_1

In [14]:
# Much faster!
dataset_1 = dataset.map(lower_case, batched=True)
dataset_1

Map:   0%|          | 0/423200 [00:00<?, ? examples/s]

Map:   0%|          | 0/36800 [00:00<?, ? examples/s]

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['date', 'description', 'amount', 'type', 'labels'],
        num_rows: 423200
    })
    validation: Dataset({
        features: ['date', 'description', 'amount', 'type', 'labels'],
        num_rows: 36800
    })
    test: Dataset({
        features: ['date', 'description', 'amount', 'type', 'labels'],
        num_rows: 40000
    })
})

In [15]:
dataset_1 = dataset_1.map(add_description_length, batched=True)
dataset_1

Map:   0%|          | 0/423200 [00:00<?, ? examples/s]

Map:   0%|          | 0/36800 [00:00<?, ? examples/s]

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['date', 'description', 'amount', 'type', 'labels', 'description_length'],
        num_rows: 423200
    })
    validation: Dataset({
        features: ['date', 'description', 'amount', 'type', 'labels', 'description_length'],
        num_rows: 36800
    })
    test: Dataset({
        features: ['date', 'description', 'amount', 'type', 'labels', 'description_length'],
        num_rows: 40000
    })
})

In [16]:
print(dataset_1.get("train")[0])

print(dataset_1.get("validation")[0])

print(dataset_1.get("test")[0])

{
    'date': '2022-05-06',
    'description': 'motunrayo titilope fayomi-williams/a',
    'amount': 3000.0,
    'type': 'C',
    'labels': 4,
    'description_length': 36
}

{
    'date': '2022-04-22',
    'description': 'trf/payment for ndk/frm dasman panan maan to  daniel gbasika adeboye - 011',
    'amount': 12026.8798828125,
    'type': 'D',
    'labels': 1,
    'description_length': 74
}

{
    'date': '2021-12-14',
    'description': 'atm cash wdl rou',
    'amount': 3000.0,
    'type': 'D',
    'labels': 0,
    'description_length': 16
}

In [17]:
# Drop descriptions that are less than 10
THRESHOLD: int = 10
dataset_1 = dataset_1.filter(lambda x: x.get("description_length") >= THRESHOLD)
dataset_1

Filter:   0%|          | 0/423200 [00:00<?, ? examples/s]

Filter:   0%|          | 0/36800 [00:00<?, ? examples/s]

Filter:   0%|          | 0/40000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['date', 'description', 'amount', 'type', 'labels', 'description_length'],
        num_rows: 408029
    })
    validation: Dataset({
        features: ['date', 'description', 'amount', 'type', 'labels', 'description_length'],
        num_rows: 35476
    })
    test: Dataset({
        features: ['date', 'description', 'amount', 'type', 'labels', 'description_length'],
        num_rows: 38555
    })
})

In [18]:
print(dataset_1.get("validation")[0])

{
    'date': '2022-04-22',
    'description': 'trf/payment for ndk/frm dasman panan maan to  daniel gbasika adeboye - 011',
    'amount': 12026.8798828125,
    'type': 'D',
    'labels': 1,
    'description_length': 74
}

In [19]:
def concat_data(example: dict[str, Any]) -> dict[str, Any]:
    """This is used to create a new column by combining columns in the dataset."""

    result: dict[str, Any] = {
        "body": [
            f"{date} \n {type} \n {amount} \n {description}"
            for (date, type, amount, description) in zip(
                example.get("date"),
                example.get("type"),
                example.get("amount"),
                example.get("description"),
            )
        ]
    }
    return result

In [20]:
dataset_1 = dataset_1.map(concat_data, batched=True)
dataset_1

Map:   0%|          | 0/408029 [00:00<?, ? examples/s]

Map:   0%|          | 0/35476 [00:00<?, ? examples/s]

Map:   0%|          | 0/38555 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['date', 'description', 'amount', 'type', 'labels', 'description_length', 'body'],
        num_rows: 408029
    })
    validation: Dataset({
        features: ['date', 'description', 'amount', 'type', 'labels', 'description_length', 'body'],
        num_rows: 35476
    })
    test: Dataset({
        features: ['date', 'description', 'amount', 'type', 'labels', 'description_length', 'body'],
        num_rows: 38555
    })
})

In [21]:
print(dataset_1.get("train")[0])

{
    'date': '2022-05-06',
    'description': 'motunrayo titilope fayomi-williams/a',
    'amount': 3000.0,
    'type': 'C',
    'labels': 4,
    'description_length': 36,
    'body': '2022-05-06 \n C \n 3000.0 \n motunrayo titilope fayomi-williams/a'
}

### Tokenize Data

In [22]:
from transformers import AutoTokenizer


MODEL_CHECKPOINT: str = "bert-base-cased"
tokenizer: AutoTokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)


def tokenize_function(examples: dict[str, Any]) -> dict[str, Any]:
    """This is used to tokenize the text. It returns a dict containing the
    input_ids, token_type_ids and attention_mask."""
    return tokenizer(examples.get("body"), truncation=True)

In [28]:
FEATURES: list[str] = [
    "date",
    "description",
    "amount",
    "type",
    "description_length",
    "body",
]

tokenized_dataset: DatasetDict = dataset_1.map(
    tokenize_function,
    batched=True,
    remove_columns=FEATURES,
)
tokenized_dataset

Map:   0%|          | 0/408029 [00:00<?, ? examples/s]

Map:   0%|          | 0/35476 [00:00<?, ? examples/s]

Map:   0%|          | 0/38555 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408029
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 35476
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 38555
    })
})

In [29]:
print(tokenized_dataset.get("train")[0])

{
    'labels': 4,
    'input_ids': [
        101,
        17881,
        1477,
        118,
        4991,
        118,
        5037,
        140,
        11975,
        119,
        121,
        182,
        3329,
        3488,
        6447,
        1186,
        189,
        17030,
        13200,
        1162,
        175,
        4164,
        18882,
        118,
        1209,
        18331,
        1116,
        120,
        170,
        102
    ],
    'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
}

In [30]:
# Number of unique labels
N_LABELS: int = len(tokenized_dataset.get("train").unique("labels"))

N_LABELS

11

In [31]:
from transformers import DataCollatorWithPadding


data_collator: DataCollatorWithPadding = DataCollatorWithPadding(tokenizer=tokenizer)

In [35]:
# Apply the data collator
batch: torch.Tensor = data_collator([tokenized_dataset["train"][i] for i in range(2)])

print(batch)

{
    'labels': tensor([4, 0]),
    'input_ids': tensor([[  101, 17881,  1477,   118,  4991,   118,  5037,   140, 11975,   119,
           121,   182,  3329,  3488,  6447,  1186,   189, 17030, 13200,  1162,
           175,  4164, 18882,   118,  1209, 18331,  1116,   120,   170,   102],
        [  101, 17881,  1477,   118,  5187,   118,  1955,   141, 26177,  1568,
           119,   121,   185,  2155,   120,  5127,  4779,   187,  6094,   102,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]]),
    'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0]]),
    'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0]])
}

In [37]:
# The 2nd idx has been padded with 0s
for row in batch.get("input_ids"):
    print(row)

tensor([  101, 17881,  1477,   118,  4991,   118,  5037,   140, 11975,   119,
          121,   182,  3329,  3488,  6447,  1186,   189, 17030, 13200,  1162,
          175,  4164, 18882,   118,  1209, 18331,  1116,   120,   170,   102])

tensor([  101, 17881,  1477,   118,  5187,   118,  1955,   141, 26177,  1568,
          119,   121,   185,  2155,   120,  5127,  4779,   187,  6094,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0])